In [35]:
import queue
import json
import sys
import csv
import re
import bs4
import util

In [3]:
starting_url = ("http://www.classes.cs.uchicago.edu/archive/2015/winter"
                    "/12200-1/new.collegecatalog.uchicago.edu/index.html")

limiting_domain = "classes.cs.uchicago.edu"

In [33]:
request = util.get_request(starting_url)
requeset_text = util.read_request(request)

soup = bs(requeset_text)
urls_on_intro = soup.find_all("a")
links = set()
for link in urls_on_intro:
    href = link.get('href')
    if href:
        absolute_url = util.convert_if_relative_url(starting_url, href)
        if util.is_url_ok_to_follow(absolute_url, limiting_domain):
            links.add(absolute_url)
list(links)

['http://www.classes.cs.uchicago.edu/archive/2015/winter/12200-1/new.collegecatalog.uchicago.edu/thecollege/programsofstudy.1.html',
 'http://www.classes.cs.uchicago.edu/archive/2015/winter/12200-1/new.collegecatalog.uchicago.edu/thecollege/thecurriculum.1.html',
 'http://www.classes.cs.uchicago.edu/archive/2015/winter/12200-1/new.collegecatalog.uchicago.edu/thecollege/preparationforprofessionalstudy/index.html',
 'http://www.classes.cs.uchicago.edu/archive/2015/winter/12200-1/new.collegecatalog.uchicago.edu/index.html',
 'http://www.classes.cs.uchicago.edu/archive/2015/winter/12200-1/new.collegecatalog.uchicago.edu/thecollege/jointdegreeprograms/index.html',
 'http://www.classes.cs.uchicago.edu/archive/2015/winter/12200-1/new.collegecatalog.uchicago.edu/thecollege/introduction/index.html',
 'http://www.classes.cs.uchicago.edu/archive/2015/winter/12200-1/new.collegecatalog.uchicago.edu/thecollege/researchopportunities/index.html',
 'http://www.classes.cs.uchicago.edu/archive/2015/winte

In [69]:
def process_page(url):
    
    request = util.get_request(url)
    if request is None:
        return []
    
    html = util.read_request(request)
    if not html:
        return []
    soup = bs4.BeautifulSoup(html, 'html.parser')
    return soup

def extract_links(soup, current_url, limiting_domain = 1000):
    links = set()
   
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            absolute_url = util.convert_if_relative_url(current_url, href)
            if absolute_url and util.is_url_ok_to_follow(absolute_url, limiting_domain):
                links.add(absolute_url)
    
    return links

In [72]:
starting_url = ("http://www.classes.cs.uchicago.edu/archive/2015/winter"
                "/12200-1/new.collegecatalog.uchicago.edu/index.html")
limiting_domain = "classes.cs.uchicago.edu"

# YOUR CODE HERE
url_queue = queue.Queue()
visited_urls = set(starting_url)

link_hierarchy = {
    1: set(),
    2: set()
    }
lv1_soup = process_page(starting_url)
lv1_links = extract_links(lv1_soup, starting_url, limiting_domain)
link_hierarchy[1].update(lv1_links)
for url in lv1_links:
    visited_urls.add(url)
    lv2_soup = process_page(url)
    lv2_links = extract_links(lv2_soup, url, limiting_domain)
    for lv2_link in lv2_links:
        url_queue.put(lv2_link)
    link_hierarchy[2].update(lv2_links) # for tracking

while url_queue and len(visited_urls) <= 100:
    item = url_queue.get()
    visited_urls.add(item)
    if item:
        process_page(item)

In [73]:
url_queue.get()

'http://www.classes.cs.uchicago.edu/archive/2015/winter/12200-1/new.collegecatalog.uchicago.edu/thecollege/jointdegreestat.1.html'

In [ ]:
antho = 'https://www.classes.cs.uchicago.edu/archive/2015/winter/12200-1/new.collegecatalog.uchicago.edu/thecollege/anthropology/index.html'


In [ ]:
def extract_course_codes(title_text):
    """
    Extracts all course codes from a title text, including sequences of any length.
    
    This function handles cases like:
    - Single courses: "CMSC 12100"
    - Two-course sequences: "ANTH 20701-20702"
    - Multi-course sequences: "CMSC 12100-12200-12300-12400"
    
    Args:
        title_text (str): The course title text containing course codes
        
    Returns:
        list: A list of all course codes found in the text
    """

    pattern = r'((?:\d{5}(?:-\d{5})*))'
    matches = re.finditer(pattern, title_text)
    course_codes = []

    for match in matches:
        dept = match.group(1)  # Capture the department code (e.g., "CMSC")
        all_groups = match.groups()
        print(all_groups)
        for num in all_groups[1:]:
            if num:
                print(num)
        
    return course_codes

In [ ]:
extract_course_codes('ANTH 23101-23102-23103. Introduction to Latin American Civilization I-II-III.')

In [ ]:
extract_course_codes('CMSC 12100-12200-12300-12400')

In [ ]:
def process_page(url):
    
    request = util.get_request(url)
    if request is None:
        return []
    
    html = util.read_request(request)
    if not html:
        return []
    soup = bs4.BeautifulSoup(html, 'html.parser')
    return soup

def extract_links(soup, current_url, limiting_domain):
    links = set()
   
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            absolute_url = util.convert_if_relative_url(current_url, href)
            if absolute_url and util.is_url_ok_to_follow(absolute_url, limiting_domain):
                links.add(absolute_url)
    
    return links

In [ ]:
def get_valid_words(text):
    """
    Extracts valid words from text, handling all text processing rules.
    
    This function encapsulates all our word processing logic:
    1. Converting to lowercase
    2. Finding valid words (starts with a letter and contains only letters, digits, and/or an underscore(_))
    3. Removing stop words
    
    Args:
        text (str): The input text to process
        
    Returns:
        set: A set of valid words that aren't in INDEX_IGNORE
    """
    text = text.lower()
    words = set(re.findall(r'[a-z][a-z0-9_]*', text))
    return words - INDEX_IGNORE


In [ ]:
import queue
import json
import sys
import csv
import re
import bs4
import util

In [ ]:
with open('course_map.json', 'r') as f:
    course_map = json.load(f)

In [ ]:
word_to_courses = {}
soup = process_page('https://www.classes.cs.uchicago.edu/archive/2015/winter/12200-1/new.collegecatalog.uchicago.edu/thecollege/anthropology/index.html')

if not soup:
    result = {}

for block in soup.find_all('div', class_='courseblock main'):
    subsequences = util.find_sequence(block)

    title_text = block.find('p', class_='courseblocktitle').get_text(strip = True)

    course_codes = extract_course_codes(title_text)
    print(course_codes)

    identifiers = set()
    for code in course_codes:
        if code in course_map:
            identifiers.add(course_map[code])
    
    descrip_text = block.find('p', class_ = 'courseblockdesc').get_text(strip = True)
    all_text = title_text + " " + descrip_text

    word_list = get_valid_words(all_text)
    print('word_list: ', word_list)

    for word in word_list:
        if word not in word_to_courses:
            word_to_courses[word] = identifiers.copy()
        else:
            word_to_courses[word].update(identifiers)
        
    if subsequences:
        for subsequence in subsequences:
            sub_identifiers = set()
            sub_title = subsequence.find('p', class_='courseblocktitle').get_text(strip = True)
            sub_course_code = extract_course_codes(sub_title)[0]
            sub_identifier = course_map[sub_course_code]
            sub_identifiers.add(sub_identifier)
            sub_discrip = subsequence.find('p', class_ = 'courseblockdesc').get_text(strip = True)
            all_text = sub_title + " " + sub_discrip
            
            word_list = get_valid_words(all_text)
    
            for word in word_list:
                if word not in word_to_courses:
                    word_to_courses[word] = sub_identifiers
                else:
                    word_to_courses[word].update(sub_identifiers)




In [ ]:
INDEX_IGNORE = set(['a', 'also', 'an', 'and', 'are', 'as', 'at', 'be',
                    'but', 'by', 'course', 'for', 'from', 'how', 'i',
                    'ii', 'iii', 'in', 'include', 'is', 'not', 'of',
                    'on', 'or', 's', 'sequence', 'so', 'social', 'students',
                    'such', 'that', 'the', 'their', 'this', 'through', 'to',
                    'topics', 'units', 'we', 'were', 'which', 'will', 'with',
                    'yet'])